#### 19：图像金字塔和图像拉普拉斯金字塔 (用于图像放大和缩小)

In [ ]:
# 向下取样   -- 缩小
# 高斯卷积核：自身像素点取36，近一些的取24，越远的像素越低。最后除以像素点个数
# 删除偶数行列：使尺寸变为原始四分之一。向下取样会丢失一些信息
# dst=cv2.pyrDown(src)
import cv2
import numpy as np
down_src = cv2.imread("../../image/black.jpg")
down=cv2.pyrDown(down_src)
cv2.imshow("down",down)
# 向上取样   -- 放大
# 在每个方向上扩大到原来的2倍，新增的行和列用0补充
# 使用和向下取样同样的卷积核✖4，获得新增像素的新值
# dst=cv2.pyrUp(src)
up_src = cv2.imread("../../image/damie.jpg")
up=cv2.pyrUp(up_src)
cv2.imshow("up",up)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 拉普拉斯金字塔
# 结果=原始图像-先向下再向上
'''
==( Li = Gi- PyrUp(PyrDown(Gi)) )==
Li  拉普拉斯金字塔图像 得规则比例图片
Gi  原图像
'''
import cv2
import numpy as np
o = cv2.imread("../../image/shy.jpg")
od=cv2.pyrDown(o)
odu=cv2.pyrUp(od)
print(o.shape,odu.shape)
lapPyr0=o-odu

o1=od
o1d=cv2.pyrDown(o1)
o1du=cv2.pyrUp(o1d)
print(o1.shape,o1du.shape)
lapPyr1=o1-o1du

cv2.imshow("orginal",o)
cv2.imshow("laplas_0",lapPyr0)
cv2.imshow("laplas_1",lapPyr1)
cv2.waitKey()
cv2.destroyAllWindows()

#### 20：图像轮廓
详情解析:https://zhuanlan.zhihu.com/p/511984691
mode:https://img-blog.csdnimg.cn/20200706183507600.png
method:https://img-blog.csdnimg.cn/20200706183539500.png

In [12]:
'''
1.对象为二值图像，首先进行阈值分割或者边缘检测。
2.查找轮廓需要更改原始图像，通常使用原始图像的一份进行拷贝。
3.在opencv里,是从黑色背景里找白色。因此对象必须是白色，背景为黑色。
'''
# 通过cv2.findContours() 查找轮廓在哪里，再通过 cv2.drawContours()将查找的轮廓绘制出来。
'''
cv2.findContours()
contours,hierarchy=cv2.findContours(image,mode,method)
contours:轮廓
hierarchy:图像的拓扑信息（轮廓层次）（存储上一个轮廓，父轮廓…）
image:原始图像
mode:轮廓检索方式
method:轮廓的近似方法
'''
'''
r=cv2.drawContours(image, contours, contourIdx, color[, thickness])
r:目标图像
image:原始图像
contours: 所有的输入轮廓边缘数组
contourIdx ：需要绘制的边缘索引，如果全部绘制为-1。如果有多个目标,可以绘制第一个目标0,第二个目标1,第三个目标2.。。
color:绘制的颜色,为BGR格式的SCalar
thickness:可选，绘制的密度，即轮廓的画笔粗细
'''
import cv2
import numpy as np
o = cv2.imread('../../image/OCRTest.jpg')
o = cv2.resize(o,(375,500))
gray = cv2.cvtColor(o,cv2.COLOR_BGR2GRAY)#BGR-灰度   cv2.cvtColor(input_image, flag)用于颜色空间转换
ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)#二值图像
contours, hierarchy = cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

co=o.copy()#对原始图像进行绘制
r=cv2.drawContours(co,contours,-1,(0,0,225),2)#co为复制图像，轮廓会修改原始图像
cv2.imshow("original",o)
cv2.imshow("binary",gray)
cv2.imshow("contours",r)
cv2.waitKey()
cv2.destroyAllWindows()

轮廓特征和近似

In [8]:
import cv2
import numpy as np
o = cv2.imread('../../image/black.jpg')
gray = cv2.cvtColor(o,cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
co=o.copy()
r=cv2.drawContours(co,cnt,-1,(0,0,225),2)
cv2.imshow("res",r)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
epsilon = 0.1*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)#cv2.approxPolyDP()函数的输出，为近似多边形的顶点坐标
print(approx.shape,cnt.shape)
co =o.copy()
img1 = cv2.drawContours(co, [approx], -1, (0, 0, 255), 2)
cv2.imshow("resu",img1)
cv2.waitKey()
cv2.destroyAllWindows()

(4, 1, 2) (250, 1, 2)


In [12]:
hull = cv2.convexHull(cnt)
co =o.copy()
img = cv2.drawContours(co, [hull], 0,  (255, 0, 0), 2)
cv2.imshow("result",img1)
cv2.waitKey()
cv2.destroyAllWindows()

边界矩形

In [10]:
x,y,w,h = cv2.boundingRect(cnt)# 这个函数可以获得一个图像的最小矩形边框一些信息
co =o.copy()
img = cv2.rectangle(co,(x,y),(x+w,y+h),(0,255,0),2)# 画框的函数
cv2.imshow("result_b",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
area = cv2.contourArea(cnt)
x,y,w,h = cv2.boundingRect(cnt)
rect_area = w*h
extent = float(area) / rect_area
print('轮廓面积与边界矩形比：',extent)

外接圆

In [11]:
(x,y),radius = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
co =o.copy()
img = cv2.circle(co,center,radius,(0,255,0),2)
cv2.imshow("result_bg",img)
cv2.waitKey()
cv2.destroyAllWindows()